<a href="https://colab.research.google.com/github/Alexandre-Delplanque/TFE-2020/blob/master/Model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object detection
---
Utilisation de "[mmdetection](https://github.com/open-mmlab/mmdetection.git)". Mmdetection s'utilise normalement sur Linux (Windows n'est pas encore supporté), Colab est donc une bonne alternative pour les utilisateurs de Windows.

Au préalable, télécharger puis loader sur Google Drive, [cette version](https://github.com/open-mmlab/mmdetection/tree/c0ac99eff015c108b34a9f80e3ff59b106dbc62e) encore non-officielle du repository. Cette version permet d'utiliser l'entiereté des réseaux, sans apparition d'erreurs.

Une partie du code est prévue pour copier ce dossier de fichier vers `/content/mmdetection_update` afin d'éviter de modifier les fichiers sources originales dans le drive.




# Connection à "My Drive"

In [0]:
# Connection à mon drive
from google.colab import drive
drive.mount('/content/drive')

# URL vers le(s) repo(s) et définition des chemins d'accès


In [0]:
# --------------------------------------------------------
# URL vers le(s) repo(s) et définition des chemins d'accès
# --------------------------------------------------------
import os
from os.path import exists, join, basename, splitext
import shutil

# URL du repo de la data
git_repo_data = 'https://github.com/Alexandre-Delplanque/TFE-2020.git' 

# Chemin d'accès vers mmdetection
%cd /content
data_dir = os.path.abspath(splitext(basename(git_repo_data))[0])
print("Data path : {}".format(data_dir))

# Chemin d'accès vers mmdetection
mmdetection_dir = '/content/mmdetection_update'
print("MMdetection path : {}".format(mmdetection_dir))

# Copie de ce tree dans '/content' afin de préserver le dossier source
mmdetection_drive_dir = '/content/drive/My Drive/MMDetection - Object detection/mmdetection-c0ac99eff015c108b34a9f80e3ff59b106dbc62e'
shutil.copytree(mmdetection_drive_dir, mmdetection_dir)

# Modification de 'mmdetection/mmdet/models/plugins/non_local.py'
Permet de ne plus obtenir l'erreur :

```
RuntimeError: cuDNN error: CUDNN_STATUS_NOT_SUPPORTED. This error may appear if you passed in a non-contiguous input.
```

Solution trouvée [ici](https://github.com/open-mmlab/mmdetection/issues/2109)

In [0]:
# ---------------------------------------------------------------
# Modification de 'mmdetection/mmdet/models/plugins/non_local.py'
# ---------------------------------------------------------------

# Modification de la ligne 110 du fichier 'non_local.py', source de l'erreur
non_local_file = os.path.join(mmdetection_dir, 'mmdet/models/plugins/non_local.py')

with open(non_local_file) as f:
    s = f.read()

new_s = s.splitlines()
new_s[109] = '        y = y.permute(0, 2, 1).contiguous().reshape(n, self.inter_channels, h, w)'
new_s = '\n'.join(new_s)

with open(non_local_file, 'w') as f:
    f.write(new_s)

# Clone du(des) repo(s), installations et construction de MMdetection

Le message ```Restart runtime ``` va apparaitre à la fin de l'éxecution de cette cellule, cliquer sur le bouton dans la console pour redémarrer l'environnement d'exécution.

Ensuite, réexécuter les cellules précédentes (sauf la première) et celle-ci avant de continuer.

In [0]:
# -------------------------------------------------------------------
# Clone du(des) repo(s), installations et construction de MMdetection
# -------------------------------------------------------------------

# Clone du repo contenant la data
!git clone $git_repo_data

# Installation de MMcv
!pip install -q mmcv terminaltables

# Installation de MMDetection
!cd {mmdetection_dir} && python setup.py install
!pip install -r {os.path.join(mmdetection_dir, "requirements.txt")}

# Ajout du path au système
import sys
sys.path.append(mmdetection_dir)

# Effacement de l'output
from google.colab import output
output.clear()

# Création d'un dataset

In [0]:
#--------------------------------------
# Création d'un fichier 'my_dataset.py'
#--------------------------------------

# Noms des classes
class_names = ["Bubale","Buffalo","Hippopotamus","Kob","Topi","Warthog","Waterbuck"]

# Création
with open(os.path.join(mmdetection_dir, "mmdet/datasets/my_dataset.py"),'w') as f:
    f.write('from .coco import CocoDataset\n')
    f.write('from .registry import DATASETS\n')
    f.write('\n')
    f.write('@DATASETS.register_module\n')
    f.write('class MyDataset(CocoDataset): \n')
    f.write('\n')
    f.write('\tCLASSES = ({})'.format(", ".join(["\'{}\'".format(name) for name in class_names])))

# Ajout dans '__init__.py'
with open(os.path.join(mmdetection_dir, "mmdet/datasets/__init__.py"),'r') as f:
    init_content = f.read()

with open(os.path.join(mmdetection_dir, "mmdet/datasets/__init__.py"),'w') as f:
    f.write('from .my_dataset import MyDataset \n')
    f.write(init_content[:-3] + ', \'MyDataset\'\n]')


# Création et transfert des sous-images (sub-frames) vers 'mmdet/data'

In [0]:
#---------------------------------------
# 'mmdet/data'
#---------------------------------------
''' Organisation du dossier cible

mmdetection
    |---data
          |---coco
                |---annotations
                |---train
                |---val
                |---test

'''
from shutil import copyfile
import numpy as np
from os import path
import glob

# Création des dossiers de destination
ann_mmdet_dir = os.path.join(mmdetection_dir,'data/coco/annotations')
train_mmdet_dir = os.path.join(mmdetection_dir,'data/coco/train')
val_mmdet_dir = os.path.join(mmdetection_dir,'data/coco/val')
test_mmdet_dir = os.path.join(mmdetection_dir,'data/coco/test')

if path.exists(ann_mmdet_dir) is not True:
    os.makedirs(ann_mmdet_dir)
if path.exists(train_mmdet_dir) is not True:
    os.makedirs(train_mmdet_dir)
if path.exists(val_mmdet_dir) is not True:
    os.makedirs(val_mmdet_dir)
if path.exists(test_mmdet_dir) is not True:
    os.makedirs(test_mmdet_dir)

# Création des sub-frames dans les dossiers
!pip install parse
output.clear()

# Train
!python '/content/TFE-2020/Pre-processing/subframes_maker.py' \
    --img-path '/content/TFE-2020/Images/train' \
    --ann-path '/content/TFE-2020/Annotations-COCO/train_cocotype.json' \
    --out-dir '/content/mmdetection_update/data/coco/train' \
    --ann-type bbox \
    --size 500x500

# Validation
!python '/content/TFE-2020/Pre-processing/subframes_maker.py' \
    --img-path '/content/TFE-2020/Images/validation' \
    --ann-path '/content/TFE-2020/Annotations-COCO/val_cocotype.json' \
    --out-dir '/content/mmdetection_update/data/coco/val' \
    --ann-type bbox \
    --size 500x500

# Test
!python '/content/TFE-2020/Pre-processing/subframes_maker.py' \
    --img-path '/content/TFE-2020/Images/test' \
    --ann-path '/content/TFE-2020/Annotations-COCO/test_cocotype.json' \
    --out-dir '/content/mmdetection_update/data/coco/test' \
    --ann-type bbox \
    --size 500x500

In [0]:
# Déplacement des fichiers d'annotations
for folder in ['train','val','test']:
    old_name = '/content/mmdetection_update/data/coco/'+folder+'/coco_subframes.json'
    new_name = '/content/mmdetection_update/data/coco/annotations/'+folder+"_cocotype.json"
    shutil.copy(old_name, new_name)

# Regroupement des classes

In [0]:
# ------------------------
# Regroupement des classes
# ------------------------
import re

# Class perso
os.chdir('/content/TFE-2020/Pre-processing')
from COCOmgmt import COCOmgmt

# Instanciation
COCO_train = COCOmgmt('/content/mmdetection_update/data/coco/annotations/train_cocotype.json')
COCO_test = COCOmgmt('/content/mmdetection_update/data/coco/annotations/test_cocotype.json')

# Inputs 
categories = [
              {"supercategory":'animal', "id":1, "name": 'Ancelaphinae'},
              {"supercategory":'animal', "id":2, "name": 'Buffalo'},
              {"supercategory":'animal', "id":3, "name": 'Hippopotamus'},
              {"supercategory":'animal', "id":4, "name": 'Kob'},
              {"supercategory":'animal', "id":5, "name": 'Warthog'},
              {"supercategory":'animal', "id":6, "name": 'Waterbuck'}
              ]
group = {1:[1,5], 2:[2], 3:[3], 4:[4], 5:[6], 6:[7]}
output_path = '/content/mmdetection_update/data/coco/annotations/test_group_cocotype.json'
COCO_test.groupcat(categories, group, output_path)
output_path = '/content/mmdetection_update/data/coco/annotations/train_group_cocotype.json'
COCO_train.groupcat(categories, group, output_path)

# Vérification graphique
COCOgroup = COCOmgmt(output_path)
train_grp = COCOgroup.displaycatdist()

# Modification du nom des classes dans 'my_dataset.py'
class_names = []
for cat in categories:
    class_names.append(cat['name'])

dataset_file = os.path.join(mmdetection_dir, "mmdet/datasets/my_dataset.py")
with open(dataset_file) as f:
    s = f.read()
    s = re.sub('CLASSES = \(.*?\)',
               'CLASSES = ({})'.format(str(", ".join(["\'{}\'".format(name) for name in class_names]))),s)

# Ecriture dans le fichier
with open(dataset_file, 'w') as f:
    f.write(s)

# Pondération des classes
Nécessité de modifier les paramètres de la fonction de coût ([Issue #996](https://github.com/open-mmlab/mmdetection/issues/996)) : [cross_entropy](https://pytorch.org/docs/stable/nn.functional.html#cross-entropy).

In [0]:
# -----------------------
# Pondération des classes
# -----------------------
import re

n_train = [int(w) for w in np.array(train_grp)[1:,2]]

loss_file = '/content/mmdetection_update/mmdet/models/losses/cross_entropy_loss.py'
with open(loss_file) as f:
    s = f.read()

new_s = s.splitlines()
new_s[8] = 'def cross_entropy(pred, label, weight=class_weights, reduction=\'mean\', avg_factor=None):'
new_s[7] = 'n_train = {} \n\
n_train = torch.FloatTensor(n_train) \n\
class_weights = torch.min(n_train)/n_train \n\
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") \n\
class_weights = class_weights.to(device) \n'.format(n_train)

new_s = '\n'.join(new_s)

with open(loss_file, 'w') as f:
    f.write(new_s)

# Paramètres

In [0]:
#----------------------------------------------
# Configuration des modèles voulant être testés
#-----------------------------------------------

MODELS_CONFIG = {
    'libra_faster_rcnn_r50_fpn_1x': {
        'config_file': 'configs/libra_rcnn/libra_faster_rcnn_r50_fpn_1x.py'
    },
    'libra_faster_rcnn_r101_fpn_1x': {
        'config_file': 'configs/libra_rcnn/libra_faster_rcnn_r101_fpn_1x.py',
    },
    'faster_rcnn_r50_fpn_1x': {
        'config_file': 'configs/faster_rcnn_r50_fpn_1x.py',
    }
}

#--------------------------
# Paramètres d'entrainement
#--------------------------

# Sélection du modèle
selected_model = 'faster_rcnn_r50_fpn_1x'

# Nom du fichier de configurations
config_file = MODELS_CONFIG[selected_model]['config_file']
print(config_file)

# Taille des images
img_scale = (500,500)

# Nombre d'epochs pour l'entrainement
total_epochs = 15

# Learning rate
lr = 0.01
lr_steps = [5,8,12]

# Weight decay
decay = 0.0005

# Taille d'un batch
batch_size = 4


# Modification du fichier de configuration

In [0]:
import os
import re

# Chemin d'accès vers le fichier
config_fname = os.path.join(mmdetection_dir, config_file)
# S'assurer que le chemin existe
assert os.path.isfile(config_fname), '`{}` not exist'.format(config_fname)
print(config_fname)

#-------------------
# Modifications
#-------------------
fname = config_fname
float_num = '[+-]?[0-9]+\.[0-9]+'
with open(fname) as f:
    s = f.read()
    work_dir = re.findall(r"work_dir = \'(.*?)\'", s)[0]

    # Modification du nombre de classes
    s = re.sub('num_classes=.*?,',
               'num_classes={},'.format(len(class_names) + 1), s)
    # Modification du nombre d'epochs
    s = re.sub('total_epochs = \d+',
               'total_epochs = {}'.format(total_epochs), s)
    # Modification du type de dataset
    s = re.sub("dataset_type = 'CocoDataset'",
               "dataset_type = 'MyDataset'", s)
    # Modification des chemin d'accès vers les annotations
    s = re.sub("annotations/instances_train2017.json",
               "annotations/train_group_cocotype.json", s)
    s = re.sub("annotations/instances_val2017.json",
               "annotations/test_group_cocotype.json", s)
    # Modification des chemin d'accès vers les images
    s = re.sub("train2017", "train", s)
    s = re.sub("val2017", "test", s)
    # Modification allocations GPU
    s = re.sub("imgs_per_gpu=\d+",
               "imgs_per_gpu={}".format(batch_size), s)
    s = re.sub("workers_per_gpu=2",
               "workers_per_gpu=1", s)

    s = re.sub("lr={}".format(float_num),
               "lr={}".format(lr),s)
    s = re.sub("step=\[.*?\]",
               "step={}".format(str(lr_steps)),s)
    s = re.sub("weight_decay={}".format(float_num),
               "weight_decay={}".format(decay),s)
    s = re.sub("img_scale=\(.*?\)",
               "img_scale={}".format(str(img_scale)),s)

# Vérification
print(s)

In [0]:
# Ecriture dans le fichier
with open(fname, 'w') as f:
    f.write(s)

# Entrainement

In [0]:
# Reconstruction de mmdetection
%cd {mmdetection_dir}
!python setup.py develop

# Entrainement
!python tools/train.py {config_fname}

In [0]:
# Recherche du fichier checkpoint
checkpoint_file = os.path.join(mmdetection_dir, work_dir, "latest.pth")
assert os.path.isfile(checkpoint_file), '`{}` not exist'.format(checkpoint_file)

print("Chemin d\'accès : {}".format(checkpoint_file))

# Téléchargement des paramètres entrainés (checkpoint_file) et du fichier de 
# configuration (config_file)

# from google.colab import files

# files.download(config_file)
# files.download(checkpoint_file)

# Test et évaluation
[Documentation](https://mmdetection.readthedocs.io/en/latest/GETTING_STARTED.html#inference-with-pretrained-models)

Bug 1 : `TypeError: object of type 'DataContainer' has no len()`
* [Solution](https://github.com/open-mmlab/mmdetection/issues/1501)



In [0]:
# ---------------------------------------------------------------------------
# Modification du fichier 'cocoeval.py' afin de pouvoir utiliser la librairie
# ---------------------------------------------------------------------------

import re 

# Chemin d'accès vers le fichier cocoeval.py
fname = "/usr/local/lib/python3.6/dist-packages/pycocotools/cocoeval.py"

with open(fname) as f:
    # Lecture du fichier
    s = f.read()
    # Remplacement des lignes de code
    s = re.sub('self.iouThrs = (.+)',
               'self.iouThrs = np.linspace(.5, 0.95, (np.round((0.95 - .5) / .05) + 1).astype(np.int), endpoint=True)', s)
    s = re.sub('self.recThrs = (.+)',
               'self.recThrs = np.linspace(.0, 1.00, (np.round((1.00 - .0) / .01) + 1).astype(np.int), endpoint=True)', s)

# Ecriture dans le fichier
with open(fname, 'w') as f:
    f.write(s)

In [0]:
# ----
# Test
# ----
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

%cd {mmdetection_dir}
!python tools/test.py {config_fname} {checkpoint_file} --eval bbox

# Visualisation des prédictions

In [0]:
# -----------------------------
# Visualisation des prédictions
# -----------------------------

# Installation de mmdet
!pip install -q mmdet

import time
import matplotlib
import matplotlib.pylab as plt

%cd {mmdetection_dir}
import mmdet
from mmdet.models import build_detector
from mmdet.apis import inference_detector, init_detector

# Construction du modèle à partir du fichier de configuration et du fichier
# checkpoint
model = init_detector(config_fname, checkpoint_file)

# Test d'une image et visualisation des prédictions
# img_path = '/content/mmdetection_update/data/coco/test/S_07_05_16_DSC00421_S64.JPG'
# img_path = '/content/mmdetection_update/data/coco/test/S_07_05_16_DSC00402_S84.JPG'
img_path = '/content/mmdetection_update/data/coco/test/S_07_05_16_DSC00558_S25.JPG'
# img_path = '/content/mmdetection_update/data/coco/test/S_07_05_16_DSC00522_S24.JPG'
# img_path = '/content/mmdetection_update/data/coco/test/E2V2_DSC06109_S9.JPG'
# img_path = '/content/mmdetection_update/data/coco/test/S_07_05_16_DSC00381_S0.JPG'

result = inference_detector(model, img_path)

# Fonction d'affichage perso
os.chdir('/content/TFE-2020/Post-processing')
import mmdet_utils

# ---
predictions = result
img_path = img_path
coco_path = '/content/mmdetection_update/data/coco/annotations/test_group_cocotype.json'
score_thresh = 0.50
# ---

mmdet_utils.display(predictions, img_path, coco_path, score_thresh=score_thresh)
matches = mmdet_utils.match(predictions, img_path, coco_path, 0.25, 0.50)
report = mmdet_utils.report(matches, coco_path)

# Matrice de confusion sur jeu de test

In [0]:
# ----------------------------
# Matrice de confusion globale
# ----------------------------
!pip install -q mmdet
%cd {mmdetection_dir}
import mmdet
from mmdet.models import build_detector
from mmdet.apis import inference_detector, init_detector
import glob
import numpy as np

# Fonctions perso
os.chdir('/content/TFE-2020/Post-processing')
import mmdet_utils

# Modèle
model = init_detector(config_fname, checkpoint_file)

# Annotations
coco_path = '/content/mmdetection_update/data/coco/annotations/test_group_cocotype.json'

# Seuils
IoU_tresh = 0.25
score_tresh = 0.50

# List des images
img_dir = '/content/mmdetection_update/data/coco/test'
os.chdir(img_dir)
images = glob.glob('*.JPG')

i = 0
for image in images:

    img_path = os.path.join(img_dir, image)

    predictions = inference_detector(model, img_path)

    res = mmdet_utils.match(predictions, 
                                img_path,
                                coco_path, 
                                IoU_tresh, 
                                score_tresh)
    if i == 0:
        matches = res
    else:
        matches = np.concatenate((matches, res))
    
    i += 1

# Matrice de confusion
matrix = mmdet_utils.matrix(matches)
print(' ')
print(matrix)
print(' ')

# Métriques
report = mmdet_utils.report(matches, coco_path)

In [0]:
# Enregistrement des résultats sur le drive
import json
save_path = '/content/drive/My Drive/MMDetection - Object detection'

# Report
with open (os.path.join(save_path,'REPORT_FASTER50_15_W.json'),'w') as json_file:
    json.dump(report,json_file)

# Matrice de confusion
names = ['Background'] + class_names
i = 0
j = 0
confusion_matrix = {}
for name in names:
    dic = {name : {}}
    for name_bis in names:
        dic[name].update({name_bis : float(matrix[i][j])})
        confusion_matrix.update(dic)
        j += 1
    j = 0
    i += 1

with open (os.path.join(save_path,'MATRIX_FASTER50_15_W.json'),'w') as json_file:
    json.dump(confusion_matrix,json_file)